# Triathlon Data - giving context

### context
This datas are giving a context to the relevance of our porject.
Triathlon is the combination of the three endurance sports: swim - bike - run and is one of the most growing discipline of the last 30 years.
First mentioned in the 20th of 19th century in France as "les trois sports" it got forgotten for decades.
Only in 1978 it was reinvented in Hawaii where the anual Ironman still can be named as the mother of this sport.

### aim
We have two combineable datasets - one with all events between 1983 and 2022 and the other one with the individual athletes' results.
We want to show the internationality of both, the events and the aethletes by mapping it on a worldmap.

### process
after checking the data for nulls and duplicats we bring the columns to meaningfull datatypes, droping columns like person names, to make it more anonymous and prepare the data for visualisation.

We faced one major problem, as the column, that should show country-codes didn't follow a readable code (ISO-3166-1 alpha3) but was in a sport-related convention (GER instead of DEU).
By filtering for those not fitting code, we builded a dictionary from geopandas to change those codes

### results / further analysis & visualisations
In this notebook are some first visualisations for the worldmap with geopanda. But the most will happen in tableau.
As this Dataset for us was only an add-on, at first we didn't go to deep into potential questions like male vs. female distribution or difference between countries/regions that could lead to questions of potential social correlations.

In [170]:
import pandas as pd
import matplotlib as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# 1. read the data

In [171]:
df_events_raw = pd.read_csv('../data_capestone/Triathlon/public_tristat_events.csv')


In [172]:
df_stats_raw = pd.read_csv('../data_capestone/Triathlon/public_tristat_stat.csv')

# 2. EDA

## sports events

In [173]:
df_events_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   event_dt     6012 non-null   object
 1   event_flg    6012 non-null   object
 2   event_name   6012 non-null   object
 3   event_f_cnt  6012 non-null   int64 
 4   event_m_cnt  6012 non-null   int64 
 5   event_link   6012 non-null   object
dtypes: int64(2), object(4)
memory usage: 281.9+ KB


no NULLs

In [174]:
df_events_raw.head()

,event_dt,event_flg,event_name,event_f_cnt,event_m_cnt,event_link
0,2022-12-11,USA,Ironman Florida 70.3 2022,383,1105,/rus/result/ironman/florida/half/2022
1,2022-12-10,NZL,Ironman New Zealand 2022,93,318,/rus/result/ironman/new-zealand/full/2022
2,2022-12-10,NZL,Ironman New Zealand 70.3 2022,275,532,/rus/result/ironman/new-zealand/half/2022
3,2022-12-09,BRN,Ironman Bahrain 70.3 2022,107,576,/rus/result/ironman/bahrain/half/2022
4,2022-12-04,ARG,Ironman Argentina 2022,21,290,/rus/result/ironman/argentina/full/2022


## sports athletes

In [175]:
df_stats_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2961502 entries, 0 to 2961501
Data columns (total 12 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   event_link                     object
 1   gender                         object
 2   person_link                    object
 3   person_flag                    object
 4   person_name                    object
 5   person_event_group             object
 6   person_event_swim_time_text    object
 7   person_event_t1_time_text      object
 8   person_event_cycle_time_text   object
 9   person_event_t2_time_text      object
 10  person_event_run_time_text     object
 11  person_event_finish_time_text  object
dtypes: object(12)
memory usage: 271.1+ MB


Due to the amount of rows the .info() doesn't work well to look for NaN

In [176]:
# are there an NaN?

df_stats_raw.isnull().any()

event_link                       False
gender                            True
person_link                       True
person_flag                       True
person_name                       True
person_event_group                True
person_event_swim_time_text      False
person_event_t1_time_text        False
person_event_cycle_time_text     False
person_event_t2_time_text        False
person_event_run_time_text       False
person_event_finish_time_text    False
dtype: bool

In [177]:
# what is the precentage of NaN per row

df_stats_raw.isnull().mean() * 100

event_link                       0.000000
gender                           0.000169
person_link                      2.253654
person_flag                      1.993515
person_name                      0.000034
person_event_group               0.052237
person_event_swim_time_text      0.000000
person_event_t1_time_text        0.000000
person_event_cycle_time_text     0.000000
person_event_t2_time_text        0.000000
person_event_run_time_text       0.000000
person_event_finish_time_text    0.000000
dtype: float64

In [178]:
df_stats_raw.head()

,event_link,gender,person_link,person_flag,person_name,person_event_group,person_event_swim_time_text,person_event_t1_time_text,person_event_cycle_time_text,person_event_t2_time_text,person_event_run_time_text,person_event_finish_time_text
0,/rus/result/ironman/ireland-cork/full/2019,M,/irl/profile/halliwell-mark,IRL,"Halliwell, Mark",M45-49,0:00,0:00,7:52:05,19:00,5:26:34,13:37:39
1,/rus/result/ironman/ireland-cork/full/2019,F,/usa/profile/harris-polly,USA,"Harris, Polly",F50-54,0:00,0:00,7:58:53,16:37,5:22:09,13:37:40
2,/rus/result/ironman/ireland-cork/full/2019,M,/fra/profile/peugeot-rodolphe,FRA,"Peugeot, Rodolphe",M25-29,0:00,0:00,7:57:05,10:40,5:30:10,13:37:55
3,/rus/result/ironman/ireland-cork/full/2019,F,/gbr/profile/crawford-lisa,GBR,"Crawford, Lisa",F50-54,0:00,0:00,7:44:47,12:51,5:40:21,13:37:59
4,/rus/result/ironman/ireland-cork/full/2019,M,/usa/profile/brumit-aaron,USA,"Brumit, Aaron",M45-49,0:00,0:00,7:12:38,34:01,5:51:50,13:38:29


# 3. cleaning

# 3.1 cleaning events

### 3.1.1 duplicates

In [179]:
duplicates_e = df_events_raw[df_events_raw.duplicated()]

print('Number of duplicats: ', len(duplicates_e))

Number of duplicats:  0


### 3.1.2 NaN

There are no NaN in events

### 3.1.3 datatypes

In [180]:
df_events = df_events_raw

In [181]:
## event_dt to date

df_events['event_dt'] = pd.to_datetime(df_events['event_dt'])

In [182]:
## object to string

df_events['event_flg'] = df_events['event_flg'].astype('string')

df_events['event_name'] = df_events['event_name'].astype('string')

df_events['event_link'] = df_events['event_link'].astype('string')

In [183]:
df_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6012 entries, 0 to 6011
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   event_dt     6012 non-null   datetime64[ns]
 1   event_flg    6012 non-null   string        
 2   event_name   6012 non-null   string        
 3   event_f_cnt  6012 non-null   int64         
 4   event_m_cnt  6012 non-null   int64         
 5   event_link   6012 non-null   string        
dtypes: datetime64[ns](1), int64(2), string(3)
memory usage: 281.9 KB


# 3.2 cleaning stats

### 3.2.1 duplicates

In [184]:
duplicates_s = df_stats_raw[df_stats_raw.duplicated(keep=False)]

print('Number of duplicats: ', len(duplicates_s))

Number of duplicats:  3112


In [185]:
duplicates_sorted = duplicates_s.sort_values(by=list(df_stats_raw.columns))
duplicates_sorted.head()

,event_link,gender,person_link,person_flag,person_name,person_event_group,person_event_swim_time_text,person_event_t1_time_text,person_event_cycle_time_text,person_event_t2_time_text,person_event_run_time_text,person_event_finish_time_text
2854713,/rus/result/challenge/european-championship-al...,F,/gbr/profile/wood-ger,GBR,Wood Ger,F60-64,1:41:53,10:07,7:06:02,7:00,5:51:31,14:56:31
2854714,/rus/result/challenge/european-championship-al...,F,/gbr/profile/wood-ger,GBR,Wood Ger,F60-64,1:41:53,10:07,7:06:02,7:00,5:51:31,14:56:31
2854689,/rus/result/challenge/european-championship-al...,F,/ned/profile/commandeur-brigitta,NED,Commandeur Brigitta,F60-64,1:26:50,12:51,5:56:29,10:04,4:40:20,12:26:31
2854690,/rus/result/challenge/european-championship-al...,F,/ned/profile/commandeur-brigitta,NED,Commandeur Brigitta,F60-64,1:26:50,12:51,5:56:29,10:04,4:40:20,12:26:31
2854675,/rus/result/challenge/european-championship-al...,F,/ned/profile/sollie-gerda-2,NED,Sollie Gerda,F60-64,1:14:57,5:28,5:58:12,4:55,4:38:46,12:02:17


Keep the first and drop the duplicates

In [186]:
df_stats = df_stats_raw.drop_duplicates(keep = 'first', ignore_index = True, inplace=False)


In [187]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2959946 entries, 0 to 2959945
Data columns (total 12 columns):
 #   Column                         Dtype 
---  ------                         ----- 
 0   event_link                     object
 1   gender                         object
 2   person_link                    object
 3   person_flag                    object
 4   person_name                    object
 5   person_event_group             object
 6   person_event_swim_time_text    object
 7   person_event_t1_time_text      object
 8   person_event_cycle_time_text   object
 9   person_event_t2_time_text      object
 10  person_event_run_time_text     object
 11  person_event_finish_time_text  object
dtypes: object(12)
memory usage: 271.0+ MB


### 3.2.2 NaN

In [188]:
# what is the precentage of NaN per row

df_stats.isnull().mean() * 100

event_link                       0.000000
gender                           0.000169
person_link                      2.254737
person_flag                      1.994563
person_name                      0.000034
person_event_group               0.052264
person_event_swim_time_text      0.000000
person_event_t1_time_text        0.000000
person_event_cycle_time_text     0.000000
person_event_t2_time_text        0.000000
person_event_run_time_text       0.000000
person_event_finish_time_text    0.000000
dtype: float64

#### gender

As it's less then 1% I'll drop the missing gender

In [189]:
df_stats = df_stats.dropna(subset=['gender'])

#### person_link

In [190]:

df_tri_noPerson=df_stats[df_stats['person_link'].isna()]
df_tri_noPerson.head()

,event_link,gender,person_link,person_flag,person_name,person_event_group,person_event_swim_time_text,person_event_t1_time_text,person_event_cycle_time_text,person_event_t2_time_text,person_event_run_time_text,person_event_finish_time_text
597,/rus/result/ironman/costa-rica/half/2019,M,NaN,NaN,"Salas corella, Roy",M40-44,53:47,2:53,3:12:38,2:44,2:10:36,6:22:38
2416,/rus/result/ironman/elsinore/half/2019,M,NaN,UKR,",",M30-34,34:36,7:40,3:10:01,3:04,1:59:14,5:54:35
5922,/rus/result/swissman/extreme-triathlon/full/2019,F,NaN,NaN,Heleen de Hooge,F,0:00,0:00,0:00,0:00,0:00,13:25:00
11929,/rus/result/ironman/luxembourg/half/2019,M,NaN,ISR,"הוכמן, קובי",M45-49,36:47,6:15,3:05:22,4:29,2:22:23,6:15:15
14337,/rus/result/celtman/extreme-scottish-triathlon...,F,NaN,NaN,Catriona Macleod,FWHITE,1:18:38,11:09,7:44:01,0:00,4:48:50,14:02:38


That would be 66.000 participants.
But do they include other information like gender or country?

In [191]:
# what other columns may be missing

df_tri_noPerson.isnull().mean() * 100

event_link                         0.000000
gender                             0.000000
person_link                      100.000000
person_flag                       88.460328
person_name                        0.001498
person_event_group                 0.555930
person_event_swim_time_text        0.000000
person_event_t1_time_text          0.000000
person_event_cycle_time_text       0.000000
person_event_t2_time_text          0.000000
person_event_run_time_text         0.000000
person_event_finish_time_text      0.000000
dtype: float64

Less then 1% missing the person_event_group. So I will drop those but keep the rest for now.

In [192]:
# droping the cases where person_link AND person_event_group are NaN

df_stats = df_stats.dropna(subset=['person_link', 'person_event_group'], how='all')


#### person_event_group

In [193]:
df_stats.isnull().mean() * 100

event_link                       0.000000
gender                           0.000000
person_link                      2.242353
person_flag                      1.982214
person_name                      0.000034
person_event_group               0.039736
person_event_swim_time_text      0.000000
person_event_t1_time_text        0.000000
person_event_cycle_time_text     0.000000
person_event_t2_time_text        0.000000
person_event_run_time_text       0.000000
person_event_finish_time_text    0.000000
dtype: float64

Less then 1% missing person_event_group, so I drop them

In [194]:
df_stats = df_stats.dropna(subset=['person_event_group'])

In [195]:
print(f"The number of rows with any NaN is {df_stats.isnull().any(axis=1).sum():,}")

The number of rows with any NaN is 66,364


I'll keep the rest for now.

### 3.2.3 datatypes

3.2.3.1. timedelta

In [196]:
## timeinterval via timedelta for all _time_text columns as _time

#  1. prepare object to HH:MM:SS / seperated by ':' => three parts 
#           if only 2 parts, add 00: as hours, fill minutes up to 0x: if needed
#           if already three parts, fill hours up to 0x: if needed

def convert_to_hhmmss(time_str):
    parts = time_str.split(':')
    if len(parts) == 2:
        minutes = parts[0].zfill(2) 
        seconds = parts[1]
        return f"00:{minutes}:{seconds}" 
    elif len(parts) == 3:
        hours = parts[0].zfill(2) 
        minutes = parts[1]
        seconds = parts[2]
        return f"{hours}:{minutes}:{seconds}"  
    return time_str 

In [197]:
## run function on the different columns

# swim

df_stats['swim_time_intervals'] = df_stats['person_event_swim_time_text'].apply(convert_to_hhmmss)

# bike

df_stats['bike_time_intervals'] = df_stats['person_event_cycle_time_text'].apply(convert_to_hhmmss)

# run

df_stats['run_time_intervals'] = df_stats['person_event_run_time_text'].apply(convert_to_hhmmss)

# t1 change

df_stats['t1_time_intervals'] = df_stats['person_event_t1_time_text'].apply(convert_to_hhmmss)

# t2 change

df_stats['t2_time_intervals'] = df_stats['person_event_t2_time_text'].apply(convert_to_hhmmss)

# finish time

df_stats['finish_time_intervals'] = df_stats['person_event_finish_time_text'].apply(convert_to_hhmmss)


In [198]:
## timeinterval via timedelta for all _time_text columns as _time

# 2. change to timedelta

# swim

df_stats['person_event_swim_time'] = pd.to_timedelta(df_stats['swim_time_intervals'])

# bike

df_stats['person_event_cycle_time'] = pd.to_timedelta(df_stats['bike_time_intervals'])

# run

df_stats['person_event_run_time'] = pd.to_timedelta(df_stats['run_time_intervals'])

# t1 change

df_stats['person_event_t1_time'] = pd.to_timedelta(df_stats['t1_time_intervals'])

# t2 change

df_stats['person_event_t2_time'] = pd.to_timedelta(df_stats['t2_time_intervals'])

# finish time

df_stats['person_event_finish_time'] = pd.to_timedelta(df_stats['finish_time_intervals'])


3.2.3.2. strings

In [199]:
# objects to strings    

df_stats['event_link'] = df_stats['event_link'].astype('string')

df_stats['gender'] = df_stats['gender'].astype('string')

df_stats['person_link'] = df_stats['person_link'].astype('string')

df_stats['person_flag'] = df_stats['person_flag'].astype('string')

df_stats['person_event_group'] = df_stats['person_event_group'].astype('string')

check the datatype changing

In [200]:
df_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2958394 entries, 0 to 2959945
Data columns (total 24 columns):
 #   Column                         Dtype          
---  ------                         -----          
 0   event_link                     string         
 1   gender                         string         
 2   person_link                    string         
 3   person_flag                    string         
 4   person_name                    object         
 5   person_event_group             string         
 6   person_event_swim_time_text    object         
 7   person_event_t1_time_text      object         
 8   person_event_cycle_time_text   object         
 9   person_event_t2_time_text      object         
 10  person_event_run_time_text     object         
 11  person_event_finish_time_text  object         
 12  swim_time_intervals            object         
 13  bike_time_intervals            object         
 14  run_time_intervals             object         
 15  t1_

### 3.2.4 drop columns

drop the person names and the columns, that we just used for changing the timedelta-datatype

In [201]:
df_stats = df_stats.drop(axis = 1,
                         columns = [
                        'person_name',
                        'person_event_swim_time_text',
                        'person_event_t1_time_text',
                        'person_event_cycle_time_text',
                        'person_event_t2_time_text',  
                        'person_event_run_time_text',
                        'person_event_finish_time_text',
                        'swim_time_intervals',
                        'bike_time_intervals',
                        'run_time_intervals',
                        't1_time_intervals',
                        't2_time_intervals',
                        'finish_time_intervals'
                         ]
                        )

In [202]:
df_stats.describe()

,person_event_swim_time,person_event_cycle_time,person_event_run_time,person_event_t1_time,person_event_t2_time,person_event_finish_time
count,2958394,2958394,2958394,2958394,2958394,2958394
mean,0 days 00:48:49.518890654,0 days 03:44:27.893521282,0 days 02:49:08.119488817,0 days 00:05:43.973725609,0 days 00:05:35.545532136,0 days 07:37:53.359474769
std,0 days 00:36:13.939063033,0 days 02:02:16.947961094,0 days 01:46:36.090354786,0 days 00:10:37.576584291,0 days 00:22:10.719305278,0 days 03:59:54.224848733
min,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0 days 00:00:10
25%,0 days 00:32:23,0 days 02:36:43,0 days 01:43:10,0 days 00:02:53,0 days 00:02:15,0 days 05:08:05
50%,0 days 00:42:10,0 days 03:09:42,0 days 02:18:35,0 days 00:04:59,0 days 00:03:50,0 days 06:23:36
75%,0 days 01:07:04,0 days 05:37:08,0 days 04:05:58,0 days 00:07:31,0 days 00:06:06,0 days 11:13:26
max,0 days 22:55:24,0 days 23:57:43,0 days 23:58:37,0 days 23:59:59,0 days 23:59:49,0 days 23:59:08


As we won't go into deeper analyses on athletes performance base, we can drop the columns with results

In [127]:
df_stats = df_stats.drop(axis = 1, columns = [
 'person_event_swim_time',
 'person_event_cycle_time',
 'person_event_run_time',
 'person_event_t1_time',
 'person_event_t2_time',
 'person_event_finish_time'])

The athletes are allready grouped by certain criteria - what groups are there?

for now, I don't go any further

# 4. First Analysis

### events: How many different countries?

In [203]:
print('Number of distinct countries: ', df_events['event_flg'].nunique())

Number of distinct countries:  132


### stats: How many unique participants?

In [205]:
print(f"Number of distinct participants: {df_stats['person_link'].nunique():,}")

Number of distinct participants: 1,021,725


### stats: How many countries are represented?

In [206]:
print('Number of distinct origin of the participants: ', df_stats['person_flag'].nunique())

Number of distinct origin of the participants:  250


# 5. First Visualisations

In [131]:
# !pip install pycountry

In [132]:
# !pip install geopandas

In [133]:
#!pip install plotly

In [207]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pycountry
import plotly.express as px

In [208]:
world = gpd.read_file('./_python_packages/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp')

PROBLEM: not all country codes follow the ISO 3166-1 alpha-3

# prep: country name

Translate the missing 50 codes to ISO

In [209]:
# Mapping the wrong country codes to ISO 3166-1 Alpha-3 Codes

corrections = {
    "VIE": "VNM", "PHI": "PHL", "RSA": "ZAF", "MAS": "MYS", "GRE": "GRC", "CRO": "HRV", "POR": "PRT", "INA": "IDN",
    "TPE": "TWN", "OMA": "OMN", "GER": "DEU", "SLO": "SVN", "PAR": "PRY", "DEN": "DNK", "LAT": "LVA", "NED": "NLD",
    "CRC": "CRI", "SUI": "CHE", "GUA": "GTM", "PUR": "PRI", "MRI": "MUS", "URU": "URY", "NEP": "NPL", "ZIM": "ZWE",
    "UAE": "ARE", "CHI": "CHL", "NGR": "NGA", "ALG": "DZA", "SAM": "WSM", "BAR": "BRB", "HON": "HND", "SKN": "KNA",
    "GRN": "GRD", "MAC": "MAC", "IVB": "VGB", "BUL": "BGR", "GIB": "GIB", "MGL": "MNG", "BAH": "BHS", "BER": "BMU",
    "SRI": "LKA", "HKG": "HKG", "ESA": "SLV", "NCA": "NIC", "ISV": "VIR", "MNP": "MNP", "IRI": "IRN", "PYF": "PYF",
    "SOL": "SLB", "FIJ": "FJI", "ARU": "ABW", "MAD": "MDG", "BOT": "BWA", "COK": "COK", "CAY": "CYM", "MON": "MCO",
    "NCL": "NCL"
}

# using the correction to column event_flg
df_events["event_flg"] = df_events["event_flg"].replace(corrections)

adding a column with country names

In [210]:
# Mapping of country code to country name + use for new column

country_mapping = {country.alpha_3: country.name for country in pycountry.countries}

## map events

In [211]:
df_events["country_name"] = df_events["event_flg"].map(country_mapping)

Count of events to show on a map

In [212]:
# Number of events by country

event_counts = df_events["event_flg"].value_counts().reset_index()
event_counts.columns = ["country", "event_count"]

In [213]:
# merge world map with df_events

world_events = world.merge(event_counts, how="left", left_on="SOVEREIGNT", right_on="country", suffixes=("", "_duplicate"))
world_events = world.drop(columns=[col for col in world.columns if "_duplicate" in col])


In [141]:
# worldmap by number of events

fig = px.choropleth(
    event_counts,
    locations="country",
    locationmode="ISO-3",
    color="event_count",
    title="Number of events per country",
    color_continuous_scale=px.colors.sequential.RdPu,
)


fig.update_layout(width=1000, height=600

' \nfig = px.choropleth(\n    event_counts,\n    locations="country",\n    locationmode="ISO-3",\n    color="event_count",\n    title="Number of events per country",\n    color_continuous_scale=px.colors.sequential.RdPu,\n)\n\n\nfig.update_layout(width=1000, height=600)\n'

## map athletes

In [214]:
# using correction to column person_flg
df_stats["person_flag"] = df_stats["person_flag"].replace(corrections)

In [215]:
# Mapping of country code to country name + use for new column

country_mapping = {country.alpha_3: country.name for country in pycountry.countries}

In [216]:
df_stats["person_country"] = df_stats["person_flag"].map(country_mapping)

Count of athletes to show on a map

In [217]:
# Number of events by country

athletes_counts = df_stats["person_flag"].value_counts().reset_index()
athletes_counts.columns = ["person_country", "athletes_count"]

In [218]:
# merge world map with df_events

world_athletes = world.merge(event_counts, how="left", left_on="SOVEREIGNT", right_on="country", suffixes=("", "_duplicate"))
world_athletes = world.drop(columns=[col for col in world.columns if "_duplicate" in col])


In [147]:
# worldmap by number of athletes

fig = px.choropleth(
    athletes_counts,
    locations="country",
    locationmode="ISO-3",
    color="athletes_count",
    title="Number of athletes per country",
    color_continuous_scale=px.colors.sequential.RdPu,
)


fig.update_layout(width=1000, height=600)


' \nfig = px.choropleth(\n    athletes_counts,\n    locations="country",\n    locationmode="ISO-3",\n    color="athletes_count",\n    title="Number of athletes per country",\n    color_continuous_scale=px.colors.sequential.RdPu,\n)\n\n\nfig.update_layout(width=1000, height=600)\n'

___________________

We stop here and load the data to the database to then do visualisation in tableau.

# SQAlchemy

In [219]:
from sqlalchemy import create_engine, types
from sqlalchemy import text # to be able to pass string
from sqlalchemy.exc import SQLAlchemyError
from dotenv import dotenv_values # to load the data from .env file

preparing connection

In [220]:
#load the data from .env file
config = dotenv_values()

# define variables for the login
pg_user = config['AZURE_USER'] 
pg_host = config['AZURE_HOST']
pg_port = config['AZURE_PORT']
pg_db = config['AZURE_DB']
pg_schema = config['AZURE_SCHEMA']
pg_pass = config['AZURE_PASS']

# build the URL
url = f'postgresql://{pg_user}:{pg_pass}@{pg_host}:{pg_port}/{pg_db}'

# create the engine
engine = create_engine(url, echo=False)
engine.url

# build the search path
my_schema = pg_schema 
with engine.begin() as conn: 
    result = conn.execute(text(f'SET search_path TO {my_schema};'))

testing the connection

In [221]:
def test_db_connection(engine):
    try:
        connection = engine.connect() # including 'connection' as variable to close the connection
        print("Connection successful!")
        connection.close() # closing the connection
        return True
    except SQLAlchemyError as e:
        print(f"Connection failed: {e}")
        return False

# Test the connection
test_db_connection(engine)

Connection successful!


True

load DataFrames to Database

## events

In [222]:
df_events.to_sql('df_tri_events_clean', con=engine, schema='public', if_exists='replace', index=False)

12

## stats

In [223]:
df_stats.to_sql('df_tri_stats_clean', con=engine, schema='public', if_exists='replace', index=False)

394